how this notebook works

In [ ]:
# dependencies

setting up a dataset

splitting the dataset

balancing the dataset

normalization / scaling

categorical values one-hot encoding

selecting a model

print model performance metrics on test data